<a href="https://colab.research.google.com/github/joangog/object-detection/blob/main/coco2017_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model comparison on COCO 2017 dataset
*   SSD300 VGG16
*   Faster R-CNN ResNet-50 FPN
*   Mask R-CNN ResNet-50 FPN
*   YOLOv5s
*   YOLOv5m






### Get requirements

In [7]:
%%shell

# Install Yolov5
cd /content
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install --quiet -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [10]:
%%shell

# Clone asset files
cd /content
git clone https://github.com/joangog/object-detection-assets
cd object-detection-assets
mv assets ../
rm -rf /content/object-detection-assets/

Cloning into 'object-detection-assets'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 2), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [9]:
cp /content/vision/references/detection/utils.py ../

### Import packages

In [1]:
# Import packages

import os
from google.colab import files

import math
import sys
import time
import numpy as np
import json
import cv2

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision.models.detection import ssd300_vgg16, fasterrcnn_resnet50_fpn, maskrcnn_resnet50_fpn
from torchvision.datasets import CocoDetection
from torchvision.transforms.functional import to_pil_image

import assets.utils
from assets.utils import collate_fn
from assets.transforms import Compose, ToTensor
from assets.coco_utils import get_coco_api_from_dataset
from assets.coco_eval import CocoEvaluator

### Define aux functions

In [2]:
# Copied and adjusted from repo pytorch/vision/references/detection/engine.py

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

@torch.no_grad()
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = assets.utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)
        targets = [sample for target in targets for sample in target]  # Added this line

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"]: output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator, outputs


### Download COCO 2017 dataset

In [7]:
%%shell

# Download images
cd /content
wget 'http://images.cocodataset.org/zips/val2017.zip'
unzip -q 'val2017.zip'
rm 'val2017.zip'

--2021-08-09 08:01:06--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.14.188
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.14.188|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  15.9MB/s    in 50s     

2021-08-09 08:01:57 (15.4 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [8]:
%%shell

# Download annotations
cd /content
wget 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
unzip -q 'annotations_trainval2017.zip'
rm 'annotations_trainval2017.zip'
cp '/content/annotations/instances_val2017.json' '/content/val2017'

--2021-08-09 08:02:03--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.110.252
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.110.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  16.3MB/s    in 16s     

2021-08-09 08:02:20 (14.8 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



### Load COCO 2017 dataset

In [3]:
img_dir = '/content/val2017'
ann_file = os.path.join(img_dir,'instances_val2017.json')  # annotations

# Define data transforms
transforms = Compose([ToTensor()]) # Compose([Resize((640,640)), ToTensor()])

# Create dataset
dataset = CocoDetection(img_dir, ann_file, transforms = transforms)

# Create data loader
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


### Load pre-trained model

In [8]:
%cd /content

# @markdown Model Selection { display-mode: 'form', run: 'auto' }
model_name = 'Mask R-CNN ResNet-50 FPN' # @param ['SSD300 VGG16', 'Faster R-CNN ResNet-50 FPN', 'Mask R-CNN ResNet-50 FPN', 'YOLOv5s', 'YOLOv5m']

if model_name == 'SSD300 VGG16':
  model_id = 'ssd'
  model = ssd300_vgg16(pretrained=True, progress=True)
elif model_name == 'Faster R-CNN ResNet-50 FPN':
  model_id = 'fasterrcnn'
  model = fasterrcnn_resnet50_fpn(pretrained=True, progress=True)
elif model_name == 'Mask R-CNN ResNet-50 FPN':
  model_id = 'maskrcnn'
  model = maskrcnn_resnet50_fpn(pretrained=True, progress=True)
elif model_name == 'YOLOv5s':
  model_id = 'yolov5s'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
elif model_name == 'YOLOv5m':
  model_id = 'yolov5m'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

print('Loaded model: '+ model_name + '\n')


/content


Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth



Loaded model: Mask R-CNN ResNet-50 FPN



### Evaluate model

In [9]:
# Get appropriate device for model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Evaluate model
evaluator, outputs = evaluate(model, data_loader, device)



Test:  [   0/2500]  eta: 0:54:30  model_time: 1.0786 (1.0786)  evaluator_time: 0.0193 (0.0193)  time: 1.3082  data: 0.2074  max mem: 1075
Test:  [ 100/2500]  eta: 0:36:14  model_time: 0.7791 (0.8475)  evaluator_time: 0.0300 (0.0500)  time: 0.8627  data: 0.0042  max mem: 1494
Test:  [ 200/2500]  eta: 0:34:33  model_time: 0.8005 (0.8447)  evaluator_time: 0.0432 (0.0495)  time: 0.8966  data: 0.0041  max mem: 1494
Test:  [ 300/2500]  eta: 0:33:04  model_time: 0.8300 (0.8444)  evaluator_time: 0.0328 (0.0503)  time: 0.9361  data: 0.0036  max mem: 1494
Test:  [ 400/2500]  eta: 0:31:41  model_time: 0.7782 (0.8481)  evaluator_time: 0.0450 (0.0503)  time: 0.8937  data: 0.0037  max mem: 1495
Test:  [ 500/2500]  eta: 0:30:07  model_time: 0.8243 (0.8466)  evaluator_time: 0.0311 (0.0503)  time: 0.9139  data: 0.0040  max mem: 1558
Test:  [ 600/2500]  eta: 0:28:41  model_time: 0.8214 (0.8489)  evaluator_time: 0.0539 (0.0506)  time: 0.9323  data: 0.0039  max mem: 1558
Test:  [ 700/2500]  eta: 0:27:12  

## Save results

In [ ]:
results = []
for i, img_id in enumerate(img_ids):  # For every image
  img_predictions = predictions[i]
  pred_boxes = img_predictions['boxes'].tolist()
  pred_labels = img_predictions['labels'].tolist()
  pred_scores = img_predictions['scores'].tolist()
  for j in range(0,len(pred_box)):  # For every predicted object
    pred_box = pred_boxes[j]
    results.append({
        'image_id': img_id,
        'category_id': pred_labels[j],
        'bbox': [round(val,1) for val in pred_box],  # Round for lower file size
        'score': pred_scores[j]
        })

results_file = os.path.join(img_dir,f'{model_id}_coco17_results.json')
with open(results_file, 'w') as outfile: 
    json.dump(results, outfile,indent = 2)



In [ ]:
files.download(results_file) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# results_dataset = dataset.loadRes(results_file)
# eval = cocoeval.COCOeval(dataset, results_dataset, iouType='bbox')
# eval.evaluate()
# eval.accumulate()
# eval.summarize()

